## IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydotplus
import seaborn as sns
import datetime
from IPython.display import Image  
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz

## FUNCTIONS

In [ ]:
def countNonNulls(array):
    return array.notnull().sum()

def padWithZero(array):
    return array.asfreq().fillna(0)

def data_x(year, data):
    year_to_slice = year
    data_year_to_slice = data[(data.date > datetime.date(year_to_slice,1,1)) &
                 (data.date < datetime.date((year_to_slice+1),1,1)) ]  
    return data_year_to_slice

def testXgBoost(predictors,target):
    dmatrix = xgb.DMatrix(data=predictors,label=target)

    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.2, random_state=1234)

    model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.6, learning_rate = 0.1,
                    max_depth = 5, alpha = 10, n_estimators = 30)

    model.fit(predictors_train,target_train)

    predictors_test.isnull().sum()
    predictors_test.shape
    predictors_test.columns.values

    #predictors_test = predictors_test.loc[~(np.isnan(predictors_test['Open']))]   
    #target_test = target_test.loc[~(np.isnan(target_test))]   


    predictions = model.predict(predictors_test)

    rmse = np.sqrt(mean_squared_error(target_test, predictions))
    print("RMSE: ",rmse)
    return model
    


# DATA ANALYSIS

## General overview of the data

The dataset can be found at: https://www.kaggle.com/mczielinski/bitcoin-historical-data.  We are going to use the dataset referring to the coinbase exchange.

In [ ]:
coinbase_data = pd.read_csv('coinbase.csv')
print(coinbase_data.head(20),"\n")

print(coinbase_data.shape,"\n")

print("Columns: ", list(coinbase_data.columns.values),"\n")

print(coinbase_data.describe(),"\n")

The dataset is composed by 8 columns, with each entry representing a time period of 1 minute:
* Timestamp - Unix timestamp, seconds elapsed since January 1, 1970
* Open - Price of first transaction of the minute. 
* High - Highest price registered for a transaction during this time period
* Low - Sames as high, but gives the lowest price
* Close - Price registered in the last transaction
* Volume_(BTC) - Total amount of BTC(bitcoin) traded.
* Volume_(Currency) - Total amount of USD traded.
* Weighted_Price - Average price of a BTC unit during the time period.

Above we can see the first 20 entries of the dataset and it's column names. 

Since we have null values, we will have to address them later. We can also see that the dataset is composed by a total of 2099760 entries.



We wish to explore both models that are and aren't aware of time. Since we'll want to run a realtime simulation, for models that aren't aware of time (such as random forests), all features will have to be shifted back by a factor of one, to simulate a real scenario were we would only have access to the OHLC(Open,High,Low,Close) and volume values of the previous time period. We'll also leave this for a later section.


# Addressing null values

 When a record as at least one null value, the remaining values are also null, except for the timestamp. We can prove this by removing all the entries where all the values, except timestamp, are null, and confirming that no null values remain.

In [ ]:
#Number of missing values
print(coinbase_data.isnull().sum(), "\n\n")

copyData = coinbase_data.copy()

print("Removing null rows...")

#Remove entries where all features, except timestamp, are null
copyData = copyData.loc[~(np.isnan(copyData['Open']) 
                                        & np.isnan(copyData['High']) 
                                        & np.isnan(copyData['Low']) 
                                        & np.isnan(copyData['Close']) 
                                        & np.isnan(copyData['Volume_(BTC)']) 
                                        & np.isnan(copyData['Volume_(Currency)']) 
                                        & np.isnan(copyData['Weighted_Price']))]

#any missing values?
print("Are there any missing values? ",copyData.isnull().values.any())

As described in the kaggle data description, this null rows represent time periods where either the exchange service was down or there were no trades being fulfilled.

Below we show a heatmap of null values and a graph of non null values per day. We can see that the amount of null values diminishes as more days go by. This is probably due to the fact that when the exchange opened the amount of trades was small. Later on, null values probably represent maintenance or failure periods of the coinbase platform, during which users couldn't trade in the exchange.
Rows were grouped by day to facilitate data visualization.

In [ ]:
%reset_selective copyData
copyData = coinbase_data.copy()

# Had date
copyData['date'] = pd.to_datetime(copyData['Timestamp'],unit='s').dt.date

#Heatmap with missing values
heatmap = copyData.copy()
heatmap.index = heatmap['date']
fig, ax = plt.subplots(figsize=(10,10))
heatmap = heatmap.iloc[:, :-1] 
sns.heatmap(heatmap.isnull(), cbar=False, ax = ax)


In [ ]:
# Dataframe with count of non null values per day
nonNullsPerDay = copyData.groupby('date', as_index=False).agg({
        'Open'  : countNonNulls
    })
print(nonNullsPerDay.head(20))
nonNullsPerDay.set_index(pd.DatetimeIndex(nonNullsPerDay['date']), inplace=True)
nonNullsPerDay = nonNullsPerDay.drop('date',axis=1)
nonNullsPerDay = nonNullsPerDay.resample('D').asfreq().fillna(0)
nonNullsPerDay = nonNullsPerDay.rename(index=str, columns={'Open': 'NonNullCount'})
print(nonNullsPerDay.head(20))
print(nonNullsPerDay.shape)



#Non null values per day
plt.plot(range(1,len(nonNullsPerDay.index)+1),nonNullsPerDay['NonNullCount'])
plt.show()

## Exploratory Analysis

Here we make a generic exploratory analysis. We see how bitcoin price varied overtime and what are the correlation levels between variables. 

In [ ]:
#Variation of bitcoin's price
plt.rcParams['figure.figsize'] = (15, 9)
copyData["Weighted_Price"].plot(grid = True)
plt.title('Price variation')


In [ ]:
#using seaborn
f,ax=plt.subplots(figsize=(15,15))
sns.heatmap(copyData.corr(), annot=True, linewidths=.5, fmt= '.2f',ax=ax)
plt.show()

In [ ]:
data_2012 = copyData[copyData.date < datetime.date(2013,1,1)] #No transactions
data_2013 = data_x(2013,copyData) #No transactions
data_2014 = data_x(2014,copyData) #27 transactions
data_2015 = data_x(2015,copyData) #507435 transactions
data_2016 = data_x(2016,copyData) #501586 transactions
data_2017 = data_x(2017,copyData) #514769 transactions
data_2018 = data_x(2018,copyData) #523827 transactions
data_2019 = data_x(2019,copyData) #9966 transactions

In [ ]:
one_dim_array = [ data_2012.shape[0], data_2013.shape[0], data_2014.shape[0],
                 data_2015.shape[0], data_2016.shape[0], data_2017.shape[0],
                 data_2018.shape[0], data_2019.shape[0]]



In [ ]:
array_years = range(2012,2019)

In [ ]:
x_pos = [i for i, _ in enumerate(one_dim_array)]

plt.rcParams['figure.figsize'] = (15, 9)
plt.bar(x_pos, one_dim_array, color='green')
plt.xlabel("Years")
plt.ylabel("Number of transactions")
plt.title("Number of transactions per year")

plt.xticks(x_pos, list(array_years))

plt.show()

# Model analysis and data preparation 

Above is an example using a gradient boosting framework, xg_boost, with an underlying tree. This first example only adjusts the values so that an entry only knows previous values. 
The gradient boosting does not understand which time it refers to in that particular moment that the prediction is being made, so it was necessary to make the shift in the values in order to fix that.

In [ ]:
coinbase_data = pd.read_csv('coinbase.csv')
coinbase_data = coinbase_data.loc[~(np.isnan(coinbase_data['Open']) 
                                        & np.isnan(coinbase_data['High']) 
                                        & np.isnan(coinbase_data['Low']) 
                                        & np.isnan(coinbase_data['Close']) 
                                        & np.isnan(coinbase_data['Volume_(BTC)']) 
                                        & np.isnan(coinbase_data['Volume_(Currency)']) 
                                        & np.isnan(coinbase_data['Weighted_Price']))]
nonTimeAwareDataset = coinbase_data.copy()
print(nonTimeAwareDataset.isnull().values.any())
#TODO: upsample removed NAN entries

#Can it predict anything with just the previous record values?
nonTimeAwareDataset = nonTimeAwareDataset.drop(['Timestamp'],axis=1)
nonTimeAwareDataset['Weighted_Price'] = nonTimeAwareDataset['Weighted_Price'].shift(-1)

predictors, target = nonTimeAwareDataset.iloc[:,:-1],nonTimeAwareDataset.iloc[:,-1]
print(predictors.dtypes)
print(target.dtypes)
print(predictors.head())
print(target.head())
model = testXgBoost(predictors,target)
#RMSE: 5002. Can't really predict anything, plot doesn't even show predictors
xgb.plot_importance(model,max_num_features=10)
plt.show()




#Maybe there is a seasonality 
#One hot encoding of month and day of month
#add year

Next we try adding previous prices, since the previous one didn't give us a usable rmse value.

In [ ]:
nonTimeAwareDataset.insert(loc = 0, column = 'laggedPriceBy1', value = nonTimeAwareDataset['Weighted_Price'].shift(1))
nonTimeAwareDataset.insert(loc = 0, column = 'laggedPriceBy2', value = nonTimeAwareDataset['Weighted_Price'].shift(2))
nonTimeAwareDataset.insert(loc = 0, column = 'laggedPriceBy3', value = nonTimeAwareDataset['Weighted_Price'].shift(3))

predictors, target = nonTimeAwareDataset.iloc[:,:-1],nonTimeAwareDataset.iloc[:,-1]

model = testXgBoost(predictors,target)
#RMSE: 5002. Can't really predict anything, plot doesn't even show predictors
xgb.plot_importance(model,max_num_features=10)
plt.show()

We still don't have an usable model, so we'll try creating a dataframe with only previous prices.

In [ ]:
coinbase_data = pd.read_csv('coinbase.csv')
coinbase_data = coinbase_data.loc[~(np.isnan(coinbase_data['Open']) 
                                        & np.isnan(coinbase_data['High']) 
                                        & np.isnan(coinbase_data['Low']) 
                                        & np.isnan(coinbase_data['Close']) 
                                        & np.isnan(coinbase_data['Volume_(BTC)']) 
                                        & np.isnan(coinbase_data['Volume_(Currency)']) 
                                        & np.isnan(coinbase_data['Weighted_Price']))]
data = {'Weighted_Price': coinbase_data['Weighted_Price'],
        'laggedPriceBy1':coinbase_data['Weighted_Price'].shift(1),
        'laggedPriceBy2':coinbase_data['Weighted_Price'].shift(2),
        'laggedPriceBy3':coinbase_data['Weighted_Price'].shift(3)}
nonTimeAwareDataset = pd.DataFrame(data, columns = ['laggedPriceBy1'
                                                    ,'laggedPriceBy2'
                                                    ,'laggedPriceBy3'
                                                    , 'Weighted_Price'])




print(nonTimeAwareDataset.isnull().values.any())
print(nonTimeAwareDataset.head())
#TODO: upsample removed NAN entries


predictors, target = nonTimeAwareDataset.iloc[:,:-1],nonTimeAwareDataset.iloc[:,-1]
print(predictors.dtypes)
print(target.dtypes)
print(predictors.head())
print(target.head())
model = testXgBoost(predictors,target)
#RMSE: 5002. Can't really predict anything, plot doesn't even show predictors
xgb.plot_importance(model,max_num_features=10)
plt.show()


## Random Forest

Below we use the ensemble learning method of random forest to generate a model,
in order to predict the weighted_price of the bitcoin in the next
period of time.

In this first instance we are merely using the package scikit-learn
to create a regressor with a random configuration, that we consider to be
a good beginning to the tests.

In [ ]:
coinbase_data = pd.read_csv('coinbase.csv')
coinbase_data = coinbase_data.loc[~(np.isnan(coinbase_data['Open']) 
                                        & np.isnan(coinbase_data['High']) 
                                        & np.isnan(coinbase_data['Low']) 
                                        & np.isnan(coinbase_data['Close']) 
                                        & np.isnan(coinbase_data['Volume_(BTC)']) 
                                        & np.isnan(coinbase_data['Volume_(Currency)']) 
                                        & np.isnan(coinbase_data['Weighted_Price']))]
nonTimeAwareDataset = coinbase_data.copy()
print(nonTimeAwareDataset.isnull().values.any())

nonTimeAwareDataset = nonTimeAwareDataset.drop(['Timestamp'],axis=1)
nonTimeAwareDataset['Weighted_Price'] = nonTimeAwareDataset['Weighted_Price'].shift(-1)
nonTimeAwareDataset.drop(nonTimeAwareDataset.tail(1).index,inplace=True)

predictors, target = nonTimeAwareDataset.iloc[:,:-1],nonTimeAwareDataset.iloc[:,-1]

print(predictors.head())
print(target.head())

initial_model_rf = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1)

predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.2, random_state=1234)

initial_model_rf.fit(predictors_train, target_train)

# Prediction of the target attribute using the test dataset of predictors.
predictions = initial_model_rf.predict(predictors_test)
rmse = np.sqrt(mean_squared_error(target_test, predictions))
print("RMSE: ", rmse)


To understand the generated model, we are going to proceed with it's analysis,
beginning with the analysis of one of the estimators generated.

In [ ]:
# Get an decision tree from the list of estimators of the model
tree = initial_model_rf.estimators_[0]

# List with the names of the features
features_list = list(predictors_train.columns.values)

# Exporting the decision tree, using the format .dot.
tree_dot = export_graphviz(tree, max_depth = 3, feature_names = features_list)

# Generating a graph of the tree from the .dot data.
graph = pydotplus.graphviz.graph_from_dot_data(tree_dot)

# Creating and presenting a png image of the graph.
Image(graph.create_png())



Since we need to improve the model, we will check the importance of the 
features used, to see witch of them is more important to the model.

In [ ]:
importances = list(initial_model_rf.feature_importances_)
features_importance = list(zip(features_list, importances))
features_importance = sorted(features_importance, key=lambda a: a[1], reverse=True)

%matplotlib inline

x_values = list(range(len(importances)))
plt.bar(x_values, importances, orientation='vertical')
plt.xticks(x_values, features_list, rotation='vertical')
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Features Importances')
plt.show()

The last method we are using in order to obtain an improve in the performance
of the model is the tunning of the hyper-parameters provided by the sklearn
random Forest algorithm.

The RandomForestRegressor has a total of 12 hyper-parameters.
Although all of those can be tuned we are only using the following subset
of hyper-parameters:

* n_estimators: the number of trees in the forest;
* criterion: the function used to measure the quality of a split;
* max_depth: the maximum depth of each tree;
* min_samples_split: the minimum number of samples required to split an internal node;
* min_samples_leaf: the minimum number of samples required to be at a leaf node;
* max_features: the number of features to consider when searching for the best split;
* bootstrap: a boolean indicating whether bootstrap samples are used when building each tree;

In [ ]:
n_estimators = [int(x) for x in np.linspace(start=10, stop=100, num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
min_samples_split = [2, 5, 7, 12]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]


random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
model_rf = RandomForestRegressor()

model_rf_random = RandomizedSearchCV(estimator = model_rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

model_rf_random.fit(predictors_train, target_train)

print(model_rf_random.best_params_)

# Prediction of the target attribute using the test dataset of predictors.
predictions = initial_model_rf.predict(predictors_test)
rmse = np.sqrt(mean_squared_error(target_test, predictions))
print("RMSE: ", rmse)
